The main solving big ridge regression problems

In [76]:
workspace()

In [77]:
using Plots

LoadError: LoadError: InitError: MethodError: no method matching rawtype(::Type{LastMain.FixedPointNumbers.FixedPoint{UInt8,8}})[0m
Closest candidates are:
  rawtype{T<:Integer,f}([1m[31m::Type{FixedPointNumbers.FixedPoint{T<:Integer,f}}[0m) at /home/ROCQ/sierra/rgower/.julia/v0.5/FixedPointNumbers/src/FixedPointNumbers.jl:95
  rawtype{F<:FixedPointNumbers.FixedPoint{T<:Integer,f}}([1m[31m::Type{F<:FixedPointNumbers.FixedPoint}[0m) at /home/ROCQ/sierra/rgower/.julia/v0.5/FixedPointNumbers/src/FixedPointNumbers.jl:96
  rawtype([1m[31m::FixedPointNumbers.FixedPoint{T<:Integer,f}[0m) at /home/ROCQ/sierra/rgower/.julia/v0.5/FixedPointNumbers/src/FixedPointNumbers.jl:97[0m
during initialization of module PlotThemes
while loading /home/ROCQ/sierra/rgower/.julia/v0.5/Plots/src/Plots.jl, in expression starting on line 10

In [225]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [226]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 10;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [227]:
dataset = "a9a"
prob = defineProb(dataset);

../data/a9a.jld


In [228]:
norm(prob.A*prob.xsol - prob.b)

4.459713089533306e-11

In [229]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [258]:
n  =20;
s = 4;
rho = convert(Int64,floor(n/s));
ind = sample(1:n,n,replace=false);
modns = mod(n,s);
divi = n-modns;
if(modns!=0)
    indM = Array{Int64}(s+1,rho);
else
    indM = Array{Int64}(s,rho);
end
    indM[1:s,:] =reshape(ind[1:divi], s,rho);
if(modns!=0)
    indM[s+1,1:modns] = ind[divi+1:end];
    indM[s+1,modns+1:end] =sample(1:n, rho-modns,replace=false);
end

In [259]:
indM

4×5 Array{Int64,2}:
 14  16  19  10  12
 20   1   8  17  18
 13  15   2   6  11
  9   7   3   4   5

In [ ]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =3;
options.maxiter=100;
options.skip_error_calculation =1;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [ ]:
n = 8;
prob =  get_randProb(n)
s = 2;
x = rand(n);
error1 = norm(x-prob.xsol);
rho = convert(Int64,floor(n/s)); #hard coded density of rows

SA = zeros(s,prob.n); 
ind = sample(1:prob.n,prob.n,replace=false);
ind = reshape(ind,s,convert(Int64,n/s))
# ind =  Array{Int64}(s,rho);#zeros(s,rho); # matrix of indices

# for i =1:s
#      ind[i,:] = sample(1:prob.n,rho,replace=false);
# end

SA = zeros(s,prob.n);
Sb = zeros(s);
for i =1:s
      SA[i,:] = sum(prob.A[ind[i,:],:],1);
      Sb[i] = sum(prob.b[ind[i,:]])
end
SAS = zeros(s,s); 
for i =1:s
     SAS[i,:] = sum(SA[:,ind[i,:]],2);
end
vect = SA*x-Sb; # 
y = SAS\vect;   # solving (S^TAS) y = (S^TAx-S^Tb)  
     
for i =1:s #adding on S^T y
   x[ind[i,:]] = x[ind[i,:]]-y[i];
end
print(ind)

error2 = norm(x-prob.xsol);
println(error1,"  vs   ",error2)

In [95]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =prob.n; # Int(floor(log2(prob.n)));
options.maxiter=100;
options.skip_error_calculation =1;
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 1936758
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 461.85  |  241.82  | 0.1668 
  2  | 10406.19  |  3347.88  | 0.2517 
  3  | 717390.11  |  192860.78  | 0.3226 


In [96]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 95.35  |  32.63  | 0.0164 
  2  | 86.54  |  8.47  | 0.0180 
  3  | 78.06  |  4.88  | 0.0180 
  4  | 73.66  |  2.31  | 0.0180 
  5  | 70.16  |  1.89  | 0.0180 
  6  | 66.91  |  1.11  | 0.0181 
  7  | 64.24  |  0.79  | 0.0181 
  8  | 62.53  |  0.63  | 0.0181 
  9  | 60.96  |  0.39  | 0.0181 
 10  | 59.65  |  0.36  | 0.0182 
 11  | 58.56  |  0.73  | 0.0182 
 12  | 58.35  |  0.25  | 0.0182 
 13  | 56.87  |  0.20  | 0.0182 
 14  | 54.87  |  0.21  | 0.0183 
 15  | 51.77  |  0.20  | 0.0183 
 16  | 49.29  |  0.19  | 0.0183 
 17  | 46.55  |  0.19  | 0.0183 
 18  | 44.39  |  0.17  | 0.0184 
 19  | 41.86  |  0.12  | 0.0184 
 20  | 39.97  |  0.14  | 0.0185 
 21  | 39.80  |  0.22  | 0.0185 
 22  | 39.51  |  0.16  | 0.0185 
 23  | 38.10  |  0.10  | 0.0185 
 24  | 36.49  |  0.08  | 0.0186 
 25  | 35.13  |  0.07  | 0.0186 
 26  | 33.29  |  0.08  | 0.0186 
 27  | 30.86  |  0.08  | 0.0186 
 28  | 28.40  |  0.08  | 0.01

In [74]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 95.35  |  32.63  | 0.0097 
  2  | 89.02  |  32.92  | 0.0250 
  3  | 87.80  |  12.41  | 0.0283 
  4  | 84.89  |  16.23  | 0.0299 
  5  | 84.17  |  7.37  | 0.0303 
  6  | 82.06  |  11.80  | 0.0311 
  7  | 81.54  |  5.59  | 0.0318 
  8  | 79.92  |  9.38  | 0.0324 
  9  | 79.51  |  4.50  | 0.0325 
 10  | 78.22  |  7.69  | 0.0326 
 11  | 77.89  |  3.73  | 0.0327 
 12  | 76.83  |  6.45  | 0.0331 
 13  | 76.56  |  3.15  | 0.0332 
 14  | 75.67  |  5.51  | 0.0333 
 15  | 75.44  |  2.70  | 0.0334 
 16  | 74.66  |  4.77  | 0.0335 
 17  | 74.46  |  2.35  | 0.0335 
 18  | 73.79  |  4.19  | 0.0336 
 19  | 73.61  |  2.08  | 0.0352 
 20  | 73.01  |  3.73  | 0.0353 
 21  | 72.85  |  1.86  | 0.0354 
 22  | 72.31  |  3.36  | 0.0355 
 23  | 72.17  |  1.68  | 0.0355 
 24  | 71.67  |  3.05  | 0.0356 
 25  | 71.54  |  1.53  | 0.0357 
 26  | 71.09  |  2.80  | 0.0357 
 27  | 70.97  |  1.41  | 0.0359 
 28  | 70.55  |  2.58  

362  | 52.05  |  0.16  | 0.1328 
363  | 52.04  |  0.08  | 0.1408 
364  | 51.99  |  0.16  | 0.1408 
365  | 51.98  |  0.08  | 0.1409 
366  | 51.94  |  0.16  | 0.1410 
367  | 51.93  |  0.08  | 0.1410 
368  | 51.89  |  0.15  | 0.1411 
369  | 51.87  |  0.08  | 0.1412 
370  | 51.83  |  0.15  | 0.1412 
371  | 51.82  |  0.08  | 0.1413 
372  | 51.78  |  0.15  | 0.1414 
373  | 51.77  |  0.08  | 0.1414 
374  | 51.72  |  0.15  | 0.1415 
375  | 51.71  |  0.08  | 0.1416 
376  | 51.67  |  0.15  | 0.1416 
377  | 51.66  |  0.08  | 0.1483 
378  | 51.62  |  0.15  | 0.1484 
379  | 51.61  |  0.08  | 0.1485 
380  | 51.56  |  0.15  | 0.1485 
381  | 51.55  |  0.08  | 0.1551 
382  | 51.51  |  0.15  | 0.1569 
383  | 51.50  |  0.08  | 0.1576 
384  | 51.46  |  0.15  | 0.1581 
385  | 51.45  |  0.08  | 0.1586 
386  | 51.40  |  0.15  | 0.1593 
387  | 51.39  |  0.08  | 0.1593 
388  | 51.35  |  0.15  | 0.1594 
389  | 51.34  |  0.08  | 0.1595 
390  | 51.30  |  0.15  | 0.1595 
391  | 51.29  |  0.08  | 0.1596 
392  | 51.

643  | 45.61  |  0.05  | 0.2193 
644  | 45.58  |  0.10  | 0.2193 
645  | 45.57  |  0.05  | 0.2194 
646  | 45.54  |  0.10  | 0.2195 
647  | 45.53  |  0.05  | 0.2196 
648  | 45.50  |  0.10  | 0.2208 
649  | 45.50  |  0.05  | 0.2208 
650  | 45.47  |  0.10  | 0.2209 
651  | 45.46  |  0.05  | 0.2210 
652  | 45.43  |  0.10  | 0.2211 
653  | 45.42  |  0.05  | 0.2214 
654  | 45.39  |  0.10  | 0.2217 
655  | 45.38  |  0.05  | 0.2220 
656  | 45.35  |  0.10  | 0.2227 
657  | 45.34  |  0.05  | 0.2230 
658  | 45.31  |  0.10  | 0.2233 
659  | 45.30  |  0.05  | 0.2236 
660  | 45.27  |  0.10  | 0.2238 
661  | 45.27  |  0.05  | 0.2241 
662  | 45.23  |  0.10  | 0.2244 
663  | 45.23  |  0.05  | 0.2245 
664  | 45.20  |  0.10  | 0.2245 
665  | 45.19  |  0.05  | 0.2246 
666  | 45.16  |  0.10  | 0.2247 
667  | 45.15  |  0.05  | 0.2247 
668  | 45.12  |  0.10  | 0.2248 
669  | 45.11  |  0.05  | 0.2249 
670  | 45.08  |  0.10  | 0.2249 
671  | 45.07  |  0.05  | 0.2250 
672  | 45.05  |  0.10  | 0.2251 
673  | 45.

978  | 39.98  |  0.07  | 0.2841 
979  | 39.98  |  0.04  | 0.2842 
980  | 39.96  |  0.07  | 0.2842 
981  | 39.95  |  0.04  | 0.2843 
982  | 39.93  |  0.07  | 0.2850 
983  | 39.92  |  0.04  | 0.2854 
984  | 39.90  |  0.07  | 0.2857 
985  | 39.89  |  0.04  | 0.2860 
986  | 39.87  |  0.07  | 0.2863 
987  | 39.86  |  0.04  | 0.2866 
988  | 39.84  |  0.07  | 0.2869 
989  | 39.83  |  0.04  | 0.2872 
990  | 39.81  |  0.07  | 0.2875 
991  | 39.80  |  0.04  | 0.2875 
992  | 39.78  |  0.07  | 0.2876 
993  | 39.78  |  0.04  | 0.2882 
994  | 39.75  |  0.07  | 0.2883 
995  | 39.75  |  0.04  | 0.2884 
996  | 39.72  |  0.07  | 0.2884 
997  | 39.72  |  0.04  | 0.2885 
998  | 39.70  |  0.07  | 0.2885 
999  | 39.69  |  0.04  | 0.2886 
1000  | 39.67  |  0.07  | 0.2887 


In [75]:
# Testing CD 
options.max_time = 10; options.restol = 10.0^(-8); options.maxiter=10000;
method_name = "CD";# gradient descent
options.sketchsize = 11#floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
  1  | 168.93  |  28.88  | 0.0001 
  2  | 192.14  |  15.36  | 0.0002 
  3  | 193.46  |  13.47  | 0.0002 
  4  | 214.79  |  11.70  | 0.0003 
  5  | 215.43  |  10.77  | 0.0003 
  6  | 211.31  |  10.61  | 0.0004 
  7  | 210.93  |  10.23  | 0.0004 
  8  | 198.93  |  10.46  | 0.0005 
  9  | 179.52  |  10.48  | 0.0005 
 10  | 187.67  |  10.15  | 0.0005 
 11  | 183.15  |  10.19  | 0.0006 
 12  | 180.80  |  10.01  | 0.0006 
 13  | 180.03  |  8.96  | 0.0007 
 14  | 163.58  |  9.68  | 0.0007 
 15  | 162.58  |  5.52  | 0.0008 
 16  | 161.54  |  4.62  | 0.0008 
 17  | 152.33  |  4.22  | 0.0009 
 18  | 152.00  |  3.32  | 0.0009 
 19  | 149.49  |  3.03  | 0.0009 
 20  | 144.58  |  3.05  | 0.0010 
 21  | 145.03  |  2.99  | 0.0011 
 22  | 147.31  |  2.97  | 0.0011 
 23  | 145.28  |  2.98  | 0.0012 
 24  | 129.39  |  2.80  | 0.0012 
 25  | 129.13  |  2.74  | 0.0012 
 26  | 130.21  |  2.56  | 0.0013 
 27  | 124.55  |  2.54  |

319  | 64.76  |  0.01  | 0.0103 
320  | 64.77  |  0.01  | 0.0104 
321  | 64.77  |  0.01  | 0.0104 
322  | 64.77  |  0.02  | 0.0105 
323  | 64.77  |  0.02  | 0.0106 
324  | 64.77  |  0.02  | 0.0106 
325  | 64.78  |  0.02  | 0.0107 
326  | 64.78  |  0.02  | 0.0107 
327  | 64.79  |  0.01  | 0.0107 
328  | 64.79  |  0.01  | 0.0108 
329  | 64.78  |  0.01  | 0.0108 
330  | 64.78  |  0.01  | 0.0108 
331  | 64.79  |  0.01  | 0.0108 
332  | 64.79  |  0.01  | 0.0109 
333  | 64.79  |  0.01  | 0.0109 
334  | 64.79  |  0.01  | 0.0110 
335  | 64.79  |  0.01  | 0.0110 
336  | 64.77  |  0.01  | 0.0110 
337  | 64.77  |  0.01  | 0.0111 
338  | 64.78  |  0.01  | 0.0111 
339  | 64.78  |  0.01  | 0.0111 
340  | 64.79  |  0.01  | 0.0111 
341  | 64.78  |  0.01  | 0.0112 
342  | 64.79  |  0.01  | 0.0112 
343  | 64.79  |  0.01  | 0.0112 
344  | 64.79  |  0.01  | 0.0113 
345  | 64.79  |  0.01  | 0.0113 
346  | 64.79  |  0.01  | 0.0113 
347  | 64.79  |  0.01  | 0.0114 
348  | 64.79  |  0.01  | 0.0114 
349  | 64.

575  | 64.83  |  0.01  | 0.0303 
576  | 64.83  |  0.01  | 0.0303 
577  | 64.83  |  0.01  | 0.0304 
578  | 64.83  |  0.01  | 0.0304 
579  | 64.82  |  0.01  | 0.0305 
580  | 64.83  |  0.01  | 0.0305 
581  | 64.83  |  0.01  | 0.0305 
582  | 64.83  |  0.01  | 0.0306 
583  | 64.83  |  0.01  | 0.0306 
584  | 64.83  |  0.01  | 0.0306 
585  | 64.83  |  0.01  | 0.0306 
586  | 64.83  |  0.01  | 0.0307 
587  | 64.83  |  0.01  | 0.0307 
588  | 64.83  |  0.01  | 0.0307 
589  | 64.83  |  0.01  | 0.0308 
590  | 64.83  |  0.01  | 0.0308 
591  | 64.84  |  0.01  | 0.0308 
592  | 64.83  |  0.01  | 0.0308 
593  | 64.83  |  0.01  | 0.0309 
594  | 64.84  |  0.01  | 0.0309 
595  | 64.84  |  0.01  | 0.0309 
596  | 64.84  |  0.01  | 0.0309 
597  | 64.84  |  0.01  | 0.0310 
598  | 64.84  |  0.01  | 0.0310 
599  | 64.84  |  0.01  | 0.0310 
600  | 64.84  |  0.01  | 0.0311 
601  | 64.85  |  0.01  | 0.0311 
602  | 64.85  |  0.01  | 0.0311 
603  | 64.85  |  0.01  | 0.0312 
604  | 64.85  |  0.01  | 0.0312 
605  | 64.

895  | 64.85  |  0.00  | 0.0390 
896  | 64.86  |  0.00  | 0.0390 
897  | 64.85  |  0.00  | 0.0391 
898  | 64.85  |  0.00  | 0.0391 
899  | 64.86  |  0.00  | 0.0391 
900  | 64.86  |  0.00  | 0.0391 
901  | 64.86  |  0.00  | 0.0392 
902  | 64.86  |  0.00  | 0.0392 
903  | 64.86  |  0.00  | 0.0392 
904  | 64.86  |  0.00  | 0.0392 
905  | 64.86  |  0.00  | 0.0393 
906  | 64.86  |  0.00  | 0.0393 
907  | 64.86  |  0.00  | 0.0393 
908  | 64.86  |  0.00  | 0.0393 
909  | 64.86  |  0.00  | 0.0393 
910  | 64.86  |  0.00  | 0.0394 
911  | 64.86  |  0.00  | 0.0394 
912  | 64.86  |  0.00  | 0.0394 
913  | 64.86  |  0.00  | 0.0394 
914  | 64.86  |  0.00  | 0.0395 
915  | 64.86  |  0.00  | 0.0395 
916  | 64.86  |  0.00  | 0.0395 
917  | 64.86  |  0.00  | 0.0395 
918  | 64.87  |  0.00  | 0.0396 
919  | 64.86  |  0.00  | 0.0396 
920  | 64.87  |  0.00  | 0.0396 
921  | 64.87  |  0.00  | 0.0396 
922  | 64.87  |  0.00  | 0.0396 
923  | 64.87  |  0.00  | 0.0397 
924  | 64.87  |  0.00  | 0.0397 
925  | 64.

1268  | 64.85  |  0.00  | 0.0478 
1269  | 64.85  |  0.00  | 0.0479 
1270  | 64.85  |  0.00  | 0.0479 
1271  | 64.85  |  0.00  | 0.0480 
1272  | 64.85  |  0.00  | 0.0480 
1273  | 64.85  |  0.00  | 0.0481 
1274  | 64.85  |  0.00  | 0.0481 
1275  | 64.85  |  0.00  | 0.0481 
1276  | 64.85  |  0.00  | 0.0481 
1277  | 64.85  |  0.00  | 0.0482 
1278  | 64.85  |  0.00  | 0.0482 
1279  | 64.85  |  0.00  | 0.0482 
1280  | 64.85  |  0.00  | 0.0482 
1281  | 64.85  |  0.00  | 0.0483 
1282  | 64.85  |  0.00  | 0.0483 
1283  | 64.85  |  0.00  | 0.0483 
1284  | 64.85  |  0.00  | 0.0483 
1285  | 64.85  |  0.00  | 0.0484 
1286  | 64.84  |  0.00  | 0.0484 
1287  | 64.85  |  0.00  | 0.0484 
1288  | 64.85  |  0.00  | 0.0484 
1289  | 64.85  |  0.00  | 0.0485 
1290  | 64.85  |  0.00  | 0.0485 
1291  | 64.85  |  0.00  | 0.0485 
1292  | 64.85  |  0.00  | 0.0486 
1293  | 64.85  |  0.00  | 0.0486 
1294  | 64.84  |  0.00  | 0.0486 
1295  | 64.85  |  0.00  | 0.0486 
1296  | 64.85  |  0.00  | 0.0487 
1297  | 64.85 

...

8536  | 65.08  |  0.00  | 0.2480 
8537  | 65.08  |  0.00  | 0.2480 
8538  | 65.08  |  0.00  | 0.2480 
8539  | 65.08  |  0.00  | 0.2481 
8540  | 65.08  |  0.00  | 0.2481 
8541  | 65.08  |  0.00  | 0.2482 
8542  | 65.08  |  0.00  | 0.2482 
8543  | 65.08  |  0.00  | 0.2482 
8544  | 65.08  |  0.00  | 0.2483 
8545  | 65.08  |  0.00  | 0.2483 
8546  | 65.08  |  0.00  | 0.2483 
8547  | 65.08  |  0.00  | 0.2483 
8548  | 65.08  |  0.00  | 0.2484 
8549  | 65.08  |  0.00  | 0.2484 
8550  | 65.08  |  0.00  | 0.2484 
8551  | 65.08  |  0.00  | 0.2484 
8552  | 65.08  |  0.00  | 0.2485 
8553  | 65.08  |  0.00  | 0.2485 
8554  | 65.08  |  0.00  | 0.2485 
8555  | 65.08  |  0.00  | 0.2485 
8556  | 65.08  |  0.00  | 0.2486 
8557  | 65.08  |  0.00  | 0.2486 
8558  | 65.08  |  0.00  | 0.2486 
8559  | 65.08  |  0.00  | 0.2486 
8560  | 65.08  |  0.00  | 0.2487 
8561  | 65.08  |  0.00  | 0.2487 
8562  | 65.08  |  0.00  | 0.2487 
8563  | 65.08  |  0.00  | 0.2487 
8564  | 65.08  |  0.00  | 0.2488 
8565  | 65.08 

8843  | 65.08  |  0.00  | 0.2554 
8844  | 65.08  |  0.00  | 0.2555 
8845  | 65.08  |  0.00  | 0.2555 
8846  | 65.08  |  0.00  | 0.2556 
8847  | 65.08  |  0.00  | 0.2556 
8848  | 65.08  |  0.00  | 0.2556 
8849  | 65.08  |  0.00  | 0.2557 
8850  | 65.08  |  0.00  | 0.2557 
8851  | 65.08  |  0.00  | 0.2557 
8852  | 65.08  |  0.00  | 0.2558 
8853  | 65.08  |  0.00  | 0.2558 
8854  | 65.08  |  0.00  | 0.2558 
8855  | 65.08  |  0.00  | 0.2559 
8856  | 65.08  |  0.00  | 0.2559 
8857  | 65.08  |  0.00  | 0.2559 
8858  | 65.08  |  0.00  | 0.2559 
8859  | 65.08  |  0.00  | 0.2560 
8860  | 65.08  |  0.00  | 0.2560 
8861  | 65.08  |  0.00  | 0.2560 
8862  | 65.08  |  0.00  | 0.2560 
8863  | 65.08  |  0.00  | 0.2561 
8864  | 65.08  |  0.00  | 0.2561 
8865  | 65.08  |  0.00  | 0.2561 
8866  | 65.08  |  0.00  | 0.2561 
8867  | 65.08  |  0.00  | 0.2562 
8868  | 65.08  |  0.00  | 0.2562 
8869  | 65.08  |  0.00  | 0.2562 
8870  | 65.08  |  0.00  | 0.2562 
8871  | 65.08  |  0.00  | 0.2563 
8872  | 65.08 

9153  | 65.09  |  0.00  | 0.2655 
9154  | 65.09  |  0.00  | 0.2656 
9155  | 65.09  |  0.00  | 0.2656 
9156  | 65.09  |  0.00  | 0.2656 
9157  | 65.09  |  0.00  | 0.2657 
9158  | 65.09  |  0.00  | 0.2657 
9159  | 65.09  |  0.00  | 0.2657 
9160  | 65.09  |  0.00  | 0.2658 
9161  | 65.09  |  0.00  | 0.2658 
9162  | 65.09  |  0.00  | 0.2658 
9163  | 65.09  |  0.00  | 0.2658 
9164  | 65.09  |  0.00  | 0.2659 
9165  | 65.09  |  0.00  | 0.2659 
9166  | 65.09  |  0.00  | 0.2659 
9167  | 65.09  |  0.00  | 0.2659 
9168  | 65.09  |  0.00  | 0.2660 
9169  | 65.09  |  0.00  | 0.2660 
9170  | 65.09  |  0.00  | 0.2660 
9171  | 65.09  |  0.00  | 0.2660 
9172  | 65.09  |  0.00  | 0.2661 
9173  | 65.09  |  0.00  | 0.2661 
9174  | 65.09  |  0.00  | 0.2661 
9175  | 65.09  |  0.00  | 0.2662 
9176  | 65.09  |  0.00  | 0.2662 
9177  | 65.09  |  0.00  | 0.2662 
9178  | 65.09  |  0.00  | 0.2662 
9179  | 65.09  |  0.00  | 0.2663 
9180  | 65.09  |  0.00  | 0.2663 
9181  | 65.09  |  0.00  | 0.2663 
9182  | 65.09 

9455  | 65.09  |  0.00  | 0.2730 
9456  | 65.09  |  0.00  | 0.2730 
9457  | 65.09  |  0.00  | 0.2730 
9458  | 65.09  |  0.00  | 0.2731 
9459  | 65.09  |  0.00  | 0.2731 
9460  | 65.09  |  0.00  | 0.2731 
9461  | 65.09  |  0.00  | 0.2732 
9462  | 65.09  |  0.00  | 0.2732 
9463  | 65.09  |  0.00  | 0.2732 
9464  | 65.09  |  0.00  | 0.2732 
9465  | 65.09  |  0.00  | 0.2733 
9466  | 65.09  |  0.00  | 0.2733 
9467  | 65.09  |  0.00  | 0.2733 
9468  | 65.09  |  0.00  | 0.2733 
9469  | 65.09  |  0.00  | 0.2734 
9470  | 65.09  |  0.00  | 0.2734 
9471  | 65.09  |  0.00  | 0.2734 
9472  | 65.09  |  0.00  | 0.2734 
9473  | 65.09  |  0.00  | 0.2735 
9474  | 65.09  |  0.00  | 0.2735 
9475  | 65.09  |  0.00  | 0.2735 
9476  | 65.09  |  0.00  | 0.2735 
9477  | 65.09  |  0.00  | 0.2736 
9478  | 65.09  |  0.00  | 0.2736 
9479  | 65.09  |  0.00  | 0.2736 
9480  | 65.09  |  0.00  | 0.2736 
9481  | 65.09  |  0.00  | 0.2737 
9482  | 65.09  |  0.00  | 0.2737 
9483  | 65.09  |  0.00  | 0.2737 
9484  | 65.09 

9781  | 65.09  |  0.00  | 0.2819 
9782  | 65.09  |  0.00  | 0.2819 
9783  | 65.09  |  0.00  | 0.2820 
9784  | 65.09  |  0.00  | 0.2820 
9785  | 65.09  |  0.00  | 0.2820 
9786  | 65.09  |  0.00  | 0.2820 
9787  | 65.09  |  0.00  | 0.2821 
9788  | 65.09  |  0.00  | 0.2821 
9789  | 65.09  |  0.00  | 0.2821 
9790  | 65.09  |  0.00  | 0.2822 
9791  | 65.09  |  0.00  | 0.2822 
9792  | 65.09  |  0.00  | 0.2822 
9793  | 65.09  |  0.00  | 0.2823 
9794  | 65.09  |  0.00  | 0.2823 
9795  | 65.09  |  0.00  | 0.2823 
9796  | 65.09  |  0.00  | 0.2824 
9797  | 65.09  |  0.00  | 0.2824 
9798  | 65.09  |  0.00  | 0.2824 
9799  | 65.09  |  0.00  | 0.2825 
9800  | 65.09  |  0.00  | 0.2825 
9801  | 65.09  |  0.00  | 0.2825 
9802  | 65.09  |  0.00  | 0.2825 
9803  | 65.09  |  0.00  | 0.2826 
9804  | 65.09  |  0.00  | 0.2826 
9805  | 65.09  |  0.00  | 0.2826 
9806  | 65.09  |  0.00  | 0.2826 
9807  | 65.09  |  0.00  | 0.2827 
9808  | 65.09  |  0.00  | 0.2827 
9809  | 65.09  |  0.00  | 0.2827 
9810  | 65.09 

In [ ]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [ ]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

In [ ]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];